# Ethereum transactions analysis

### Read the eth_transactions.json files

In [2]:
import os
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
PATH = '../data/'

In [11]:
pd.set_option('display.max_columns', 100, 'display.max_rows', 100, 'display.max_colwidth', 100, 'display.float_format', lambda x: '%.f' % x)

In [5]:
file_dir = os.listdir(PATH)
file_list = [os.path.join(PATH, file) for file in file_dir if file.endswith('parquet')]

In [7]:
df_raw = pd.concat([pd.read_parquet(file) for file in file_list])

In [8]:
df_raw.shape

(5493838, 20)

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5493838 entries, 0 to 2791715
Data columns (total 20 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hash                  object        
 1   blockHash             object        
 2   blockNumber           int64         
 3   from                  object        
 4   gas                   int64         
 5   gasPrice              int64         
 6   input                 object        
 7   nonce                 int64         
 8   r                     object        
 9   s                     object        
 10  to                    object        
 11  transactionIndex      int64         
 12  type                  int64         
 13  v                     int64         
 14  value                 float64       
 15  accessList            object        
 16  chainId               float64       
 17  maxFeePerGas          float64       
 18  maxPriorityFeePerGas  float64       
 19  

In [12]:
df_raw.head()

,hash,blockHash,blockNumber,from,gas,gasPrice,input,nonce,r,s,to,transactionIndex,type,v,value,accessList,chainId,maxFeePerGas,maxPriorityFeePerGas,block_timestamp
0,0xfedacdb532ec5525686557a9bca04daa357ba754ccd70bea7cf0459572c189cf,0x5502093582eef8f1c9088f8db764ab59f403787954dee81eeaf681bf8ba681a7,15064948,0xf71e4a144cda4498277f9ad89b6501ec6c83c27c,21000,33084805662,0x,535,0xdc2fccbf51c8dde791b9ec027d4d53aab9bf6d5c10f963135a1bff925203dba7,0x1c0b133e837b36281fd6bdb9abb907c2060db9f6ab1824205037f3069458cafe,0xf0fb796c4ea2f2b24939ddb20d4b66144b3ed4bc,0,0,27,1614273837860304,None,NaN,NaN,NaN,2022-07-02 19:17:48
1,0xf40f05907bfce0363bc128055d6a1270ea2469f072166c69e3358da754a0c1ba,0x5502093582eef8f1c9088f8db764ab59f403787954dee81eeaf681bf8ba681a7,15064948,0xf0fb796c4ea2f2b24939ddb20d4b66144b3ed4bc,48792,33084805662,0xa9059cbb00000000000000000000000013e68bdecfe11ae8c314307516e8b027057ddac00000000000000000000000...,181,0x3f6a162d208194c1189ef6fbc0120a5a01cba85a0a1ff06f227b6fb645832435,0x7dcd65f3728f916430683013eabea52dbae65666e134e776a4c789cd1ff53161,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1,0,27,0,None,NaN,NaN,NaN,2022-07-02 19:17:48
2,0x9f7e7d4bed78c97f3018113e620141073b6121e6cb224f4183997648e180d341,0x5502093582eef8f1c9088f8db764ab59f403787954dee81eeaf681bf8ba681a7,15064948,0x65a8f07bd9a8598e1b5b6c0a88f4779dbc077675,601375,23648988340,0x7c0252000000000000000000000000002057cfb9fd11837d61b294d514c5bd03e5e7189a0000000000000000000000...,25331,0x344ce402827c5d35864b9dd52e6211def0eacba544dd7b47d25c8f1867559b73,0x97749f02882a8740bd3b7e6f5e0abdb9e8844542fd32b3ea2d2a05ee2890063,0x1111111254fb6c44bac0bed2854e76f90643097d,2,2,0,0,[],1,250000000000,13000000000,2022-07-02 19:17:48
3,0x6e376057df8906abcb4c10196c75332925df7be7a2953bae01d052056cea80cb,0x5502093582eef8f1c9088f8db764ab59f403787954dee81eeaf681bf8ba681a7,15064948,0xcdd37ada79f589c15bd4f8fd2083dc88e34a2af2,300000,18765846322,0x79c76e1a000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,130218,0xeb9a58d4aa629a5ac8715800dc2aefbc93aa181af39017f980e9500614f97d1d,0x1ab07d951e79d495f4cb10cb5b24a2359f48bf2df8b29791439fc1429839e485,0x8fb8839a18f02a2ff72938184060b4a7d95a2e53,3,0,27,0,None,NaN,NaN,NaN,2022-07-02 19:17:48
4,0x69025f3b35b4c469bdd2a5d29f51025adff86ceb3f85dcba4e05c0bbe1ca6e69,0x5502093582eef8f1c9088f8db764ab59f403787954dee81eeaf681bf8ba681a7,15064948,0x26fd09c8b44af53df38a9bad41d5abc55a1786af,300000,10648988340,0x9a9dfe9e00000000000000000000000080c7770b4399ae22149db17e97f9fc8a10ca51000000000000000000000000...,61239,0x9cbc00b57f392f98726fddfbe1b3bc168c6a6e9868b5ed8734a261dec43900ad,0x3243874d35d959e96db7f8da059b16910723adb27751a04d0462b3ead77a67ec,0x87d9da48db6e1f925cb67d3b7d2a292846c24cf7,4,2,0,0,"[{'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'storageKeys': ['0x8fba4a48fba25cd68c...",1,19917989125,0,2022-07-02 19:17:48
